In [1]:
import numpy as np
from jetbot import Robot
from jetbot import Camera
import cv2
print(cv2.__version__)
import asyncio
from PIL import Image
from IPython.display import clear_output
import matplotlib.pyplot as plt
import ipywidgets.widgets as widgets
import traitlets
from jetbot import bgr8_to_jpeg

4.1.1


In [2]:
robot = Robot()
#camera = Camera.instance(width=1280, height=720)
#camera = Camera.instance(width=1024,height=800)
#camera = Camera.instance()
#net = cv2.dnn.readNet('data/one_sphere/yolov3.weights','data/one_sphere/yolov3.cfg' )

In [3]:
from datetime import datetime

In [4]:
from PIL import Image

In [5]:
# print(camera)

In [6]:
#PLOTTING PURPOSE
time_r = []
detections = []
counter = 0
indexer = 0

In [ ]:
from imutils.video import FileVideoStream, FPS
from queue import Queue
from threading import Thread
import time
import sys

from threading import Thread
import cv2, time




class ThreadedCamera:
    def __init__(self, path, path_out):
        self.capture = cv2.VideoCapture(path)
        self.capture.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        # FPS = 1/X
        # X = desired FPS
#         self.FPS = 1/40
        self.FPS = 1/ int((self.capture.get(cv2.CAP_PROP_FPS)))
        print("FPS delay: " + str(self.FPS))
        print("Get FPS value: " + str(int(self.capture.get(cv2.CAP_PROP_FPS))))
        self.FPS_MS = int(self.FPS * 1000)
        
        self.list_of_det = []
        self.counter = 0
        self.previous_frame = None
        self.fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        #Hardcoded width and height values, as those are being read only once the frame is captured
        self.output_video = cv2.VideoWriter(path_out, self.fourcc, int(self.capture.get(cv2.CAP_PROP_FPS)/20), (1280,720))
        self.availableFrames = True
        self.initialized = False
        # Start frame retrieval thread
        self.thread = Thread(target=self.update, args=())
        self.thread.daemon = True
        self.thread.start()

        
        
    def update(self):
        while True:
            if self.capture.isOpened():
                (self.status, self.frame) = self.capture.read()
                if self.initialized == False:
                    self.previous_frame = self.frame
                    self.initialized=True
                if self.status == False:
                    self.availableFrames = False
                    sys.exit()
                time.sleep(self.FPS)
#                 print("self.capture is opened(): " + str(time.time()))
            
    def get_frame(self):
        global counter, indexer, detections
        input_image = cv2.cvtColor(self.frame, cv2.COLOR_RGBA2RGB)
        h,w,c = input_image.shape
        print("Image input size: " + str(h) + ", " + str(w) + ", c: " + str(c))
        blob = cv2.dnn.blobFromImage(image=cv2.resize(input_image,(w,h)), scalefactor=1.0, size=(300,300), swapRB=True)
        net.setInput(blob)
        detections = net.forward()
        for j in np.arange(0, detections.shape[2]):
            confidence = detections[0,0,j,2]
            if confidence > 0.50:
                self.counter = self.counter + 1
                idx = int(detections[0,0,j,1])
                box = detections[0,0,j,3:7] * np.array([w,h,w,h])
                (startX, startY, endX, endY) = box.astype("int")
                print("Id: " + str(idx))
                label = "{}: {:.2f}%".format(CLASSES[idx], confidence*100)
                print(label)
                # Checking if the box detected is not too big -> false positive
                if endX - startX < 300:
                    cv2.rectangle(input_image, (startX,startY), (endX,endY), COLORS[idx],2)
                    y = startY - 15 if startY - 15 > 15 else startY + 15
                    cv2.putText(input_image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx],2)
                    print("Detected " + str(CLASSES[idx]) +  " at X position: " + str(startX))
                    #starting x of the detected object on the right side
                    print("startX " + str(startX))
                    print("w" + str(w/2))
#         difference = cv2.subtract(self.previous_frame, input_image)
#         if np.any(difference) and self.initialized:
        self.output_video.write(input_image)
        self.list_of_det.append(self.counter)
        self.previous_frame = input_image
        self.counter = 0
#         _, encoded_image = cv2.imencode('.png', input_image)
#         bytes_img_obj = encoded_image.tobytes()
#         gt_widget = widgets.Image(value=bytes_img_obj, format="png",width=300, height=300)
#         camera_link = traitlets.dlink((gt_widget,'value'), (image_trait, 'value'))
#         if(self.initialized == False):
#     #         clear_output(wait=True)
#             display(image_trait)
#             self.initialized = True
#         print(time.time())

In [ ]:
from jetbot import Robot
import ipywidgets.widgets as widgets

robot = Robot()
import numpy as np
import cv2
CLASSES = ["cube","cylinder","cuboid"]

COLORS = np.random.uniform(0,255,size=(len(CLASSES),3))

NET_GRAPH_PATH = 'models/'


# SSD_MOBILENETV1 trained on basic scene for 50k steps
MODEL1 = 'ssd_mobilenetv1/basic_50k_steps'
# SSD_MOBILENETV1 trained on basic scene for 100k steps
MODEL2 = 'ssd_mobilenetv1/basic_100k_steps'
# SSD_MOBILENETV2 trained on basic scene for 50k steps
MODEL3 = 'ssd_mobilenetv2/basic_50k_steps'
# SSD_INCEPTIONV2 trained on basic scene for 50k steps
MODEL4 = 'ssd_inceptionv2/basic_50k_steps'
# SSD_MOBILENETV1 trained on darker scenes (light randomization) for 50k steps
MODEL5 = 'ssd_mobilenetv1/dark_light_50k_steps'
# SSD_MOBILENETV1 trained on basic scene for 50k steps with additional background images (5%)
MODEL6 = 'ssd_mobilenetv1/basic_50k_steps_add_backgr_img'
# SSD_MOBILENETV1 trained on basic scene for 100k steps with additional background images (5%)
MODEL7 = 'ssd_mobilenetv1/basic_100k_steps_add_backgr_img'
# MODEL5 = 'ssd_mobilenetv1/ground_text'
# MODEL6 = 'ssd_mobilenetv1/multi_ground_text'
# MODEL7 = 'ssd_mobilenetv1/white_floor_tile'

# not good results
MODEL8 = 'ssd_mobilenetv1/basic_50k_steps_add_backgr_img_2k'

# SSD_MOBILENETV1 trained on an extended scene with different sizes of objects for
# 50k steps and with additional background images (5%)
MODEL9 = 'ssd_mobilenetv1/more_shapes_50k_steps_add_backgr'

# SSD_MOBILENETV1 trained on an extended scene with different sizes of objects for
# 50k steps, with additional background images (5%) and for multiple instances
MODEL10 = 'ssd_mobilenetv1/more_shapes_50k_steps_add_backgr_more_instances'

MODEL11 = 'ssd_mobilenetv1/more_shapes_50k_steps_under_wall'

MODEL12 = 'ssd_mobilenetv1/more_shapes_50k_steps_under_wall_baseboard'

# ########
# MODEL11 = 'ssd_mobilenetv1/2k_images_data/100k'
SELECTED_MODEL = MODEL12

net = cv2.dnn.readNetFromTensorflow(NET_GRAPH_PATH + SELECTED_MODEL + '/frozen_inference_graph.pb', NET_GRAPH_PATH + SELECTED_MODEL +'/final_graph.pbtxt')

VIDEO_NAME = 'drive_1_1'
VIDEO_TO_PROCESS = 'recordings_10_06/' + VIDEO_NAME + '.mp4'
VIDEO_TO_OUTPUT = 'output_video/' + SELECTED_MODEL + '/' +VIDEO_NAME + '.avi'

IMAGE_NAME = 'img_9640.png'
IMAGE_TO_OUTPUT = 'output_video/' + SELECTED_MODEL + '/' + IMAGE_NAME

import cv2
import matplotlib.pyplot as plt
image = 0
# cap = cv2.VideoCapture(VIDEO_TO_PROCESS)

In [ ]:
fvs = ThreadedCamera(VIDEO_TO_PROCESS, VIDEO_TO_OUTPUT)

image_trait = widgets.Image(format='png', width=300, height=300)

i=0
fps = FPS().start()
while fvs.availableFrames:
    try:
        fvs.get_frame()
    except AttributeError:
        pass
    except AssertionError:
        pass
fps.stop()
print("INFO: Elapsed time: {:.2f}".format(fps.elapsed()))
print("INFO: Approx FPS: {:.2f}".format(fps.fps()))
# fvs.stop()
import matplotlib.pyplot as plt
time_r = range(len(fvs.list_of_det))
plt.plot(time_r,fvs.list_of_det)
plt.xlabel('num. of processed frames')
plt.ylabel('num. of detections')
plt.show()
# PLT_TO_OUTPUT = NET_GRAPH_PATH + SELECTED_MODEL + '/output_video/' +VIDEO_NAME + 'PlotResult.jpg'
plt.savefig(PLT_TO_OUTPUT)
fvs.capture.release()
fvs.output_video.release()

Main changes applied so far: changing the layer batch optimazation FusedBatchNormV3 to FusedBatchNorm because of incongruences in tensorflow module version